## Compress DICOMs using JPEG 2000 Lossless compression
- take a large 1.9GB DICOM multi-frame image and compress it

In [0]:
%pip install --quiet numpy==1.26.4 pydicom==3.0.1 pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg

In [0]:
dbutils.library.restartPython()

In [0]:
import yaml
import pprint
cfg = yaml.safe_load(open("config.yaml", "r"))
pprint.pprint(cfg, indent=4)
input_path = cfg.get("input_path")

# Output path for the compressed DICOM file
compression = "openjpeg-HTJ2K"
output_path = (cfg.get("output_path").replace("{compression}", compression))

print(output_path)

In [0]:
import pydicom
from pydicom.uid import JPEG2000Lossless  # Choose according to your needs
import os
import time

In [0]:
start = time.time()
# Read the DICOM file
ds = pydicom.dcmread(input_path, force=False)

# Optionally, decompress if already compressed
print (ds.file_meta.TransferSyntaxUID.is_compressed)
if "TransferSyntaxUID" in ds.file_meta and ds.file_meta.TransferSyntaxUID.is_compressed:
    ds.decompress()


# Compress pixel data using JPEG 2000 (choose lossless or lossy)
# For **lossless** JPEG 2000:
ds.compress(JPEG2000Lossless)

print (ds.file_meta.TransferSyntaxUID.is_compressed)

# Save the new compressed DICOM file
ds.save_as(output_path)
duration = time.time() - start
print(f"Saved JPEG2000-compressed DICOM to {output_path}")

In [0]:
import os
output_size = os.stat(output_path).st_size
input_size = os.stat(input_path).st_size

print(f"Input size: {input_size}")
print(f"Output size: {output_size}")
print(f"Reduction:        {round(input_size/output_size,2)}x")
print(f"Savings:         {round(100*(input_size-output_size)/input_size,2)}%")
print(f"Time taken:       {round(duration,2)} seconds")

In [0]:
%sh ls -laht \
/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0.dcm \
/Volumes/douglas_moore/demo/temp/output_jpeg2000.dcm

In [0]:
import pydicom
from pydicom.uid import JPEG2000Lossless

ds = pydicom.dcmread(output_path)
ds.pixel_array_options(use_v2_backend=True)
pixel_array = ds.pixel_array  # shape: (frames, rows, cols)
pixel_array.shape